# Loading Simple File From Storage Location

In [0]:
%sql
SHOW EXTERNAL LOCATIONS;

In [0]:
%sql
LIST 'abfss://demo@dbxdl.dfs.core.windows.net/lineitems/' WITH (CREDENTIAL `dbxdl-storage-account-creds`);

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS MASTERCLASS;

In [0]:
%sql
USE CATALOG MASTERCLASS;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS BRONZE
 MANAGED LOCATION "abfss://delta@dbxdl.dfs.core.windows.net/bronze/";

In [0]:
%sql
CREATE OR REPLACE TABLE BRONZE.RAW_LINEITEMS (
  V VARIANT
) USING DELTA;

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://demo@dbxdl.dfs.core.windows.net/lineitems/*` LIMIT 5;

### IF data needs to be stored in VARIANT a Parse and struct are needed, otherwise DBX flattens the data

In [0]:
%sql
COPY INTO BRONZE.RAW_LINEITEMS
FROM (
  SELECT parse_json(to_json(struct(*))) AS V 
  FROM 'abfss://demo@dbxdl.dfs.core.windows.net/lineitems/*'
)
FILEFORMAT = PARQUET
FORMAT_OPTIONS ('singleVariantColumn' = 'true');

In [0]:
%sql
SELECT COUNT(*) FROM BRONZE.RAW_LINEITEMS;

In [0]:
%sql
SELECT
  variant_get(V, '$.L_ORDERKEY') AS order_key,
  variant_get(V, '$.L_PARTKEY') AS part_key,
  variant_get(V, '$.L_SUPPKEY') AS supp_key,
  variant_get(V, '$.L_LINENUMBER') AS line_number,
  variant_get(V, '$.L_QUANTITY') AS quantity,
  variant_get(V, '$.L_EXTENDEDPRICE') AS extended_price,
  variant_get(V, '$.L_DISCOUNT') AS discount,
  variant_get(V, '$.L_TAX') AS tax,
  variant_get(V, '$.L_RETURNFLAG') AS return_flag,
  variant_get(V, '$.L_LINESTATUS') AS line_status,
  variant_get(V, '$.L_SHIPDATE') AS ship_date,
  variant_get(V, '$.L_COMMITDATE') AS commit_date,
  variant_get(V, '$.L_RECEIPTDATE') AS receipt_date,
  variant_get(V, '$.L_SHIPINSTRUCT') AS ship_instruct,
  variant_get(V, '$.L_SHIPMODE') AS ship_mode,
  variant_get(V, '$.L_COMMENT') AS comment
FROM BRONZE.RAW_LINEITEMS
LIMIT 3;

## Autoloader DLT

In [0]:
%sql
USE CATALOG masterclass;
USE SCHEMA bronze;

-- Creating DLT
CREATE OR REFRESH STREAMING TABLE masterclass.bronze.raw_lineitems_dlt
AS
SELECT *
FROM STREAM read_files(
        'abfss://dataeng@dbxdl.dfs.core.windows.net/lineitems/',
        format          => 'parquet',
        schemaLocation  => 'abfss://delta@dbxdl.dfs.core.windows.net/_schemas/lineitems/'
);

-- Enable the variantType-preview feature on the table
-- ALTER TABLE masterclass.bronze.raw_lineitems_dlt 
-- SET TBLPROPERTIES ('delta.feature.variantType-preview' = 'supported');

In [0]:
%sql
SELECT COUNT(*) FROM masterclass.bronze.raw_lineitems_dlt;

In [0]:
%sql
-- DROP TABLE IF EXISTS masterclass.bronze.raw_lineitems_dlt;